In [5]:
from pymatgen.entries.computed_entries import ComputedEntry
import pandas
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter

from pymatgen.ext.matproj import MPRester
from pymatgen.entries.computed_entries import Composition


<h3>Get decomposition energies for compositions from MP for further benchmarking</h3>

In [82]:
df_check = pandas.read_csv('300_materials.csv')

In [84]:
def get_reference_energy(comp: Composition, el_refs) -> float:
    """Sum of elemental reference energies over all elements in a composition.

    Args:
        comp (Composition): Input composition.

    Returns:
        float: Reference energy
    """
    return sum(comp[el] * el_refs[el].energy_per_atom for el in comp.elements)

def get_entries(row):
    comp = Composition(row['composition'])
    mp_id = row['MP_id']
    elements = [str(element) for element in comp.elements]
    with MPRester("your_mp_ip") as mpr:
    # Retrieve terminal entries for individual elements from Materials Project
        # Obtain only corrected GGA and GGA+U ComputedStructureEntry objects
        entries = mpr.get_entries_in_chemsys(elements=elements,
                                             additional_criteria={"thermo_types": ["GGA_GGA+U"]}) 

    return entries

def get_MEGNET_entry(row, el_refs):
    composition = Composition(row['composition'])
    energy = row['Ef_megnet_corrected']
    my_entry = ComputedEntry(composition, 
                             energy * composition.num_atoms, 
                             correction=get_reference_energy(composition, el_refs))
    return my_entry
        


def get_index_id_material(entries, mp_id):
    return next((index for index, e in enumerate(entries) if e.entry_id == mp_id + '-GGA'), None)

def get_Hd(row):
    entries = get_entries(row)
    pd = PhaseDiagram(entries)
    mp_id = row['MP_id']
    el_refs = pd.el_refs

    Hd_MP = get_Hd_MP(entries, pd, mp_id)
    Hd_MEGNET = get_Hd_MEGNET(entries, row, mp_id, el_refs)

    return Hd_MP, Hd_MEGNET

def get_Hd_MP(entries, pd, mp_id):
    index_id = get_index_id_material(entries, mp_id)
    if index_id is not None:
        Hd_MP = pd.get_phase_separation_energy(entries[index_id], stable_only=False)
    return Hd_MP

def get_Hd_MEGNET(entries, row, mp_id, el_refs):
    entries = [e for e in entries if e.entry_id != mp_id + '-GGA']
    
    new_entry = get_MEGNET_entry(row, el_refs)
    
    entries.append(new_entry)
    
    pd_new = PhaseDiagram(entries)
    
    Hd_MEGNET = pd_new.get_phase_separation_energy(new_entry, stable_only=False)

    return Hd_MEGNET


In [85]:
df_check[['Hd_MP','Hd_MEGNET']] = df_check.apply(lambda row: get_Hd(row), axis=1, result_type="expand")


Retrieving ThermoDoc documents: 100%|██████| 77/77 [00:00<00:00, 2340300.06it/s]


Retrieving ThermoDoc documents: 100%|████| 134/134 [00:00<00:00, 3602799.59it/s]


In [86]:
df_check.to_csv('300_materials_Hd.csv')